In [1]:
import pandas as pd
import numpy as np

In [2]:
X_data = pd.read_csv("X_data.csv")
X_data = X_data.to_numpy(dtype=np.float32)

In [19]:
Y_data = pd.read_csv("Y_data.csv")
Y_data = Y_data.to_numpy(dtype=np.float32)

Пусть размерность вектора $x_1 - n_1$, $x_2 - n_2$ и $x_3 - n_3$

В случае нашей выборки #3, $n_1 = 2$, $n_2 = 2$ и $n_3 = 2$

In [31]:
n1, n2, n3 = 2, 2, 2

In [56]:
X1 = X_data[:, 0: n1]
X2 = X_data[:, n1: n1 + n2]
X3 = X_data[:, n1 + n2: n1 + n2 + n3]

Теперь нормируем входные данные относительно каждого столбца 


In [65]:
def normalize(a):
    min_element = a.min() 
    max_element = a.max() 
    return (a - min_element) / (max_element - min_element) 

In [70]:
X1_norm = np.apply_along_axis(normalize, 0, X1)
X2_norm = np.apply_along_axis(normalize, 0, X2)
X3_norm = np.apply_along_axis(normalize, 0, X3)
Y_norm = np.apply_along_axis(normalize, 0, Y_data)

In [71]:
def chebyshev_polynom_in_point(x, degree):
    values = np.zeros(degree + 1)
    values[0] = 1
    values[1] = x
    for current_degree in range(2, degree + 1):
        values[current_degree] = 2 * x * values[current_degree - 1] - values[current_degree - 2]
    
    return values[degree]

In [72]:
def star_chebyshev_polynom_in_point(x, degree):
    if degree == 0:
        return 0.5
    else:
        return chebyshev_polynom_in_point(2 * x - 1, degree)

Составим систему для нахождения $\lambda_{ij}$

Для начала зафиксируем степень смещенных полиномов Чебешева $T_{P_1}^{*}(x)$, $T_{P_2}^{*}(x)$ и $T_{P_3}^{*}(x)$:

In [74]:
P1, P2, P3 = 3, 3, 3

Количество дискретный наблюдений:

In [75]:
N = Y_norm.shape[0]

In [84]:
A = []

In [85]:
for current_sample_number in range(N):
    lambda_first = [star_chebyshev_polynom_in_point(X1_norm[current_sample_number][j1], p1) 
                    for j1 in range(n1) for p1 in range(P1 + 1)]
    
    lambda_second = [star_chebyshev_polynom_in_point(X2_norm[current_sample_number][j2], p2) 
                    for j2 in range(n2) for p2 in range(P2 + 1)]
    
    lambda_third = [star_chebyshev_polynom_in_point(X3_norm[current_sample_number][j3], p3) 
                    for j3 in range(n3) for p3 in range(P3 + 1)]
    
    current_row = np.hstack([lambda_first, lambda_second, lambda_third])
    A.append(current_row)

In [86]:
A = np.array(A)

Теперь решим нашу систему с помощью метода покоординатного спуска

In [122]:
sol = np.linalg.lstsq(A, Y_norm[:, 2])

/home/vladislav/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [123]:
sol[0]

array([ 1.32858003e-01,  8.72276710e-02, -1.24968376e-06, -9.66652643e-07,
        1.32858003e-01,  1.47199150e-01,  5.92178785e-07,  5.22772486e-08,
        1.32858003e-01,  4.14352285e-02,  1.43013834e-06,  3.73066659e-07,
        1.32858003e-01,  9.14590480e-02,  1.94303811e-02,  6.97598029e-07,
        1.32858003e-01,  2.63870347e-01,  5.01584325e-02,  1.14356014e-07,
        1.32858003e-01,  6.54233286e-02,  3.23114882e-08, -3.13004653e-07])

In [116]:
Y_norm[:, 0].shape

(40,)

In [115]:
sol[0][0]

array([0.13285788, 0.13285794, 0.132858  , 0.1328574 , 0.13285793])